# 첫번째와 두 번째 셀에 있는 crop_name 과 item_code를 바꾸고 사용하시면 됩니다.
## 결측치가 있는 행은 제거 됩니다. (ex: 2018년 1~4주차는 lag 데이터가 없어서 제거됨)
### 아이템 코드
- 양파 : 1201
- 배추 : 1001
- 상추 : 1005
- 사과 : 0601
- 무 : 1101
- 감자 : 0501
- 대파 : 1202
- 건고추 : 1207
- 마늘 : 1209
- 딸기 : 0804
- 방울토마토 : 0806
- 오이 : 0901
- 양배추 : 1004
- 고구마 : 0502
- 배 : 0602

In [205]:
import pandas as pd

crop_name = '양파'  # 여기만 '사과', '무', '양파' 등으로 바꾸면 됨

file_path = f"data/{crop_name}_이상치제거_주간기준_등급코드.csv"

df = pd.read_csv(file_path, encoding='cp949')
df_grow = pd.read_csv('data/factor_external_weekly.csv', encoding='utf-8') # 이건 고정

In [206]:
df['item_code'] = '1201'  # 거래데이터에는 대분류 코드가 없고 한 종류만 있음

df_grow['item_code'] = df_grow['item_code'].astype(str) 
df_grow['item_code'] = df_grow['item_code'].str.zfill(4)

In [207]:
def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}{week_number:02d}"

df['weekno'] = df['연월일'].apply(get_week_of_year)

In [208]:
def format_week_int(week_no):
    week_no = int(week_no)
    year = week_no // 100
    week = week_no % 100
    return f"{year}{week:02d}"

df_grow['weekno'] = df_grow['week_no'].apply(format_week_int)

In [209]:
df.drop(columns=['휴일여부', '명절지수', '작기정보'], inplace=True)

In [210]:
merged_df  = pd.merge( df,
                    df_grow[['weekno', 'item_code', 'holiday_flag', 'holiday_score', 'grow_score']],
                    left_on=['weekno', 'item_code'],
                    right_on=['weekno', 'item_code'],
                    how='left'
                )
merged_df.head()

,주차,연월일,품목코드,품목명,품종코드,품종명,등급코드,등급이름,총금액(원),총거래량(kg),...,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),item_code,weekno,holiday_flag,holiday_score,grow_score
0,2018-01-01~2018-01-07,2018-01-03,1,양파,99,기타양파,12,중,4893000.0,4200.0,...,4.1,-4.8,49.1,-9.0,-9.0,1201,201801,3,0.0,0.0
1,2018-01-01~2018-01-07,2018-01-03,1,양파,99,기타양파,12,중,9012400.0,9960.0,...,2.5,-0.4,74.1,-9.0,-9.0,1201,201801,3,0.0,0.0
2,2018-01-01~2018-01-07,2018-01-03,1,양파,99,기타양파,12,중,4245000.0,4500.0,...,2.5,-0.4,74.1,-9.0,-9.0,1201,201801,3,0.0,0.0
3,2018-01-01~2018-01-07,2018-01-03,1,양파,99,기타양파,11,고,2943000.0,2160.0,...,4.1,-4.8,49.1,-9.0,-9.0,1201,201801,3,0.0,0.0
4,2018-01-01~2018-01-07,2018-01-03,1,양파,99,기타양파,11,고,4022300.0,3024.0,...,2.5,-0.4,74.1,-9.0,-9.0,1201,201801,3,0.0,0.0


In [211]:
merged_df.loc[merged_df['강수량(mm)']<=0, '강수량(mm)'] = 0
merged_df.loc[merged_df['1시간최고강수량(mm)']<=0, '1시간최고강수량(mm)'] = 0

In [212]:
df_daily_galic = merged_df.loc[:, ['연월일', '품종코드', '등급코드', '총거래량(kg)','주간평균단가(원)','직팜산지코드','일평균기온','최고기온','최저기온','평균상대습도','강수량(mm)','1시간최고강수량(mm)', 'holiday_flag', 'holiday_score','grow_score', '총금액(원)']]
df_daily_galic.head()

,연월일,품종코드,등급코드,총거래량(kg),주간평균단가(원),직팜산지코드,일평균기온,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),holiday_flag,holiday_score,grow_score,총금액(원)
0,2018-01-03,99,12,4200.0,1016.038196,1133,-0.1,4.1,-4.8,49.1,0.0,0.0,3,0.0,0.0,4893000.0
1,2018-01-03,99,12,9960.0,1016.038196,1092,1.0,2.5,-0.4,74.1,0.0,0.0,3,0.0,0.0,9012400.0
2,2018-01-03,99,12,4500.0,1016.038196,1093,1.0,2.5,-0.4,74.1,0.0,0.0,3,0.0,0.0,4245000.0
3,2018-01-03,99,11,2160.0,1016.038196,1133,-0.1,4.1,-4.8,49.1,0.0,0.0,3,0.0,0.0,2943000.0
4,2018-01-03,99,11,3024.0,1016.038196,1092,1.0,2.5,-0.4,74.1,0.0,0.0,3,0.0,0.0,4022300.0


In [213]:
df_galic = merged_df.loc[:, ['weekno', '품종코드', '등급코드', '총금액(원)', '총거래량(kg)','직팜산지코드','일평균기온','최고기온','최저기온','평균상대습도','강수량(mm)','1시간최고강수량(mm)', 'holiday_flag', 'holiday_score','grow_score']]
df_weekly_galic = df_galic.groupby(['weekno', '품종코드', '등급코드', '직팜산지코드']).agg({
    '총금액(원)' : 'sum', 
    '총거래량(kg)' : 'sum', 
    '일평균기온': 'mean', 
    '최고기온': 'max', 
    '최저기온' : 'min', 
    '평균상대습도' : 'mean', 
    '강수량(mm)' : 'sum', 
    '1시간최고강수량(mm)' : 'sum', 
    'holiday_flag' : 'sum', 
    'holiday_score' : 'sum', 
    'grow_score' : 'sum'
}).reset_index()

In [214]:
import pandas as pd
from datetime import date  # 여기서 date만 가져옴

df_weekly_galic['평균단가(원)'] = round(df_weekly_galic['총금액(원)'] / df_weekly_galic['총거래량(kg)'])

df_weekly_galic['year'] = df_weekly_galic['weekno'].astype(str).str[:4].astype(int)
df_weekly_galic['week'] = df_weekly_galic['weekno'].astype(str).str[4:].astype(int)

def get_max_week(year):
    return date(year, 12, 28).isocalendar()[1]

df_weekly_galic = df_weekly_galic[df_weekly_galic.apply(
    lambda row: 1 <= row['week'] <= get_max_week(row['year']), axis=1
)].copy()

def year_week_to_date(year, week):
    return date.fromisocalendar(year, week, 1)  # ← 여기도 date

df_weekly_galic['week_start'] = df_weekly_galic.apply(
    lambda row: year_week_to_date(row['year'], row['week']),
    axis=1
)
df_weekly_galic['week_start'] = pd.to_datetime(df_weekly_galic['week_start'])

df_weekly_galic.drop(columns='weekno', inplace=True)

In [216]:
df_weekly_galic.to_csv('data/trade_weekly_galic.csv', encoding='cp949', index=False)

In [217]:
df_weekly_galic

,품종코드,등급코드,직팜산지코드,총금액(원),총거래량(kg),일평균기온,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),holiday_flag,holiday_score,grow_score,평균단가(원),year,week,week_start
0,0,11,1022,2430000.0,1800.0,-1.200000,3.9,-5.1,64.400000,0.0,0.0,3,0.0,0.0,1350.0,2018,1,2018-01-01
1,0,11,1079,331250.0,280.0,0.433333,7.2,-7.0,52.350000,0.0,0.0,18,0.0,0.0,1183.0,2018,1,2018-01-01
2,0,11,1148,4532000.0,3120.0,-4.166667,3.3,-15.9,48.333333,0.0,0.0,9,0.0,0.0,1453.0,2018,1,2018-01-01
3,0,12,1019,3185000.0,3640.0,-0.700000,5.6,-5.0,64.100000,0.0,0.0,3,0.0,0.0,875.0,2018,1,2018-01-01
4,0,12,1022,26302000.0,27560.0,-1.850000,6.1,-7.8,61.600000,0.0,0.0,12,0.0,0.0,954.0,2018,1,2018-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133476,99,13,1120,1065500.0,3140.0,20.366667,28.3,13.2,64.000000,0.0,0.0,6,0.0,0.0,339.0,2025,22,2025-05-26
133477,99,13,1121,204000.0,680.0,19.800000,27.2,13.1,59.500000,0.0,0.0,2,0.0,0.0,300.0,2025,22,2025-05-26
133478,99,13,1123,63000.0,180.0,17.500000,26.2,10.8,79.300000,0.6,0.6,2,0.0,0.0,350.0,2025,22,2025-05-26
133479,99,13,1144,6500.0,18.0,20.400000,28.9,11.5,62.300000,0.0,0.0,2,0.0,0.0,361.0,2025,22,2025-05-26


In [218]:
df = pd.concat([df_weekly_galic.drop(columns=['평균단가(원)', '총금액(원)']), df_weekly_galic.iloc[:, -4]], axis=1)

In [219]:
df.to_csv('data/trade_weekly_galic.csv', encoding='cp949', index=False)

# 모델링

In [220]:
import pandas as pd
import numpy as np

In [221]:
df_weekly_galic = pd.read_csv('data/trade_weekly_galic.csv', encoding='cp949')

In [222]:
print(df_weekly_galic.isna().sum())  # 대체로 산지가 없거나 수입산인 경우 기후 데이터 없음
df_weekly_galic_drop = df_weekly_galic.dropna()

품종코드                0
등급코드                0
직팜산지코드              0
총거래량(kg)            0
일평균기온            1180
최고기온             1180
최저기온             1180
평균상대습도           1180
강수량(mm)             0
1시간최고강수량(mm)        0
holiday_flag        0
holiday_score       0
grow_score          0
year                0
week                0
week_start          0
평균단가(원)             0
dtype: int64


In [223]:
df_weekly_galic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133467 entries, 0 to 133466
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   품종코드           133467 non-null  int64  
 1   등급코드           133467 non-null  int64  
 2   직팜산지코드         133467 non-null  int64  
 3   총거래량(kg)       133467 non-null  float64
 4   일평균기온          132287 non-null  float64
 5   최고기온           132287 non-null  float64
 6   최저기온           132287 non-null  float64
 7   평균상대습도         132287 non-null  float64
 8   강수량(mm)        133467 non-null  float64
 9   1시간최고강수량(mm)   133467 non-null  float64
 10  holiday_flag   133467 non-null  int64  
 11  holiday_score  133467 non-null  float64
 12  grow_score     133467 non-null  float64
 13  year           133467 non-null  int64  
 14  week           133467 non-null  int64  
 15  week_start     133467 non-null  object 
 16  평균단가(원)        133467 non-null  float64
dtypes: float64(10), int64(6), obj

In [224]:
import pandas as pd
import numpy as np

In [225]:
df = pd.read_csv('data/trade_weekly_galic.csv', encoding='cp949', parse_dates=['week_start'])
print(df.shape)
df.head()

(133467, 17)


,품종코드,등급코드,직팜산지코드,총거래량(kg),일평균기온,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),holiday_flag,holiday_score,grow_score,year,week,week_start,평균단가(원)
0,0,11,1022,1800.0,-1.200000,3.9,-5.1,64.400000,0.0,0.0,3,0.0,0.0,2018,1,2018-01-01,1350.0
1,0,11,1079,280.0,0.433333,7.2,-7.0,52.350000,0.0,0.0,18,0.0,0.0,2018,1,2018-01-01,1183.0
2,0,11,1148,3120.0,-4.166667,3.3,-15.9,48.333333,0.0,0.0,9,0.0,0.0,2018,1,2018-01-01,1453.0
3,0,12,1019,3640.0,-0.700000,5.6,-5.0,64.100000,0.0,0.0,3,0.0,0.0,2018,1,2018-01-01,875.0
4,0,12,1022,27560.0,-1.850000,6.1,-7.8,61.600000,0.0,0.0,12,0.0,0.0,2018,1,2018-01-01,954.0


In [226]:
df = df.dropna()
print(df.isna().sum())

품종코드             0
등급코드             0
직팜산지코드           0
총거래량(kg)         0
일평균기온            0
최고기온             0
최저기온             0
평균상대습도           0
강수량(mm)          0
1시간최고강수량(mm)     0
holiday_flag     0
holiday_score    0
grow_score       0
year             0
week             0
week_start       0
평균단가(원)          0
dtype: int64


In [227]:
df.head()

,품종코드,등급코드,직팜산지코드,총거래량(kg),일평균기온,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),holiday_flag,holiday_score,grow_score,year,week,week_start,평균단가(원)
0,0,11,1022,1800.0,-1.200000,3.9,-5.1,64.400000,0.0,0.0,3,0.0,0.0,2018,1,2018-01-01,1350.0
1,0,11,1079,280.0,0.433333,7.2,-7.0,52.350000,0.0,0.0,18,0.0,0.0,2018,1,2018-01-01,1183.0
2,0,11,1148,3120.0,-4.166667,3.3,-15.9,48.333333,0.0,0.0,9,0.0,0.0,2018,1,2018-01-01,1453.0
3,0,12,1019,3640.0,-0.700000,5.6,-5.0,64.100000,0.0,0.0,3,0.0,0.0,2018,1,2018-01-01,875.0
4,0,12,1022,27560.0,-1.850000,6.1,-7.8,61.600000,0.0,0.0,12,0.0,0.0,2018,1,2018-01-01,954.0


In [228]:
front_cols = ['year', 'week', 'week_start']
other_cols = [col for col in df.columns if col not in front_cols]
df = df[front_cols + other_cols]
df.reset_index(drop=True, inplace=True)
df

,year,week,week_start,품종코드,등급코드,직팜산지코드,총거래량(kg),일평균기온,최고기온,최저기온,평균상대습도,강수량(mm),1시간최고강수량(mm),holiday_flag,holiday_score,grow_score,평균단가(원)
0,2018,1,2018-01-01,0,11,1022,1800.0,-1.200000,3.9,-5.1,64.400000,0.0,0.0,3,0.0,0.0,1350.0
1,2018,1,2018-01-01,0,11,1079,280.0,0.433333,7.2,-7.0,52.350000,0.0,0.0,18,0.0,0.0,1183.0
2,2018,1,2018-01-01,0,11,1148,3120.0,-4.166667,3.3,-15.9,48.333333,0.0,0.0,9,0.0,0.0,1453.0
3,2018,1,2018-01-01,0,12,1019,3640.0,-0.700000,5.6,-5.0,64.100000,0.0,0.0,3,0.0,0.0,875.0
4,2018,1,2018-01-01,0,12,1022,27560.0,-1.850000,6.1,-7.8,61.600000,0.0,0.0,12,0.0,0.0,954.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132282,2025,22,2025-05-26,99,13,1120,3140.0,20.366667,28.3,13.2,64.000000,0.0,0.0,6,0.0,0.0,339.0
132283,2025,22,2025-05-26,99,13,1121,680.0,19.800000,27.2,13.1,59.500000,0.0,0.0,2,0.0,0.0,300.0
132284,2025,22,2025-05-26,99,13,1123,180.0,17.500000,26.2,10.8,79.300000,0.6,0.6,2,0.0,0.0,350.0
132285,2025,22,2025-05-26,99,13,1144,18.0,20.400000,28.9,11.5,62.300000,0.0,0.0,2,0.0,0.0,361.0


In [229]:
item_counts = df['품종코드'].value_counts(normalize=True).reset_index()
item_counts.columns = ['품종코드', '비율']
item_counts['누적비율'] = item_counts['비율'].cumsum()

main_items = item_counts[item_counts['누적비율'] <= 0.8]['품종코드']

df['품종코드_통합'] = df['품종코드'].apply(lambda x: x if x in main_items.values else 100)

df = df.loc[:, ~df.columns.duplicated()]

df['품종코드'] = df['품종코드_통합']

df.drop(columns=['품종코드_통합'], inplace=True)

In [230]:
df['품종코드'].unique()

array([100,   1,   2,   3,  99], dtype=int64)

In [231]:
import pandas as pd

df['yearweek'] = df['year'] * 100 + df['week']

climate_cols = [
    '일평균기온', '최고기온', '최저기온',
    '평균상대습도', '강수량(mm)', '1시간최고강수량(mm)'
]

climate_df = (
    df[['직팜산지코드', '등급코드', 'year', 'week'] + climate_cols]
    .drop_duplicates(subset=['직팜산지코드', '등급코드', 'year', 'week'])
    .copy()
)
climate_df['yearweek'] = climate_df['year'] * 100 + climate_df['week']

climate_df.sort_values(['직팜산지코드', '등급코드', 'yearweek'], inplace=True)
for lag in [1, 2, 3, 4]:
    for col in climate_cols:
        climate_df[f'{col}_t-{lag}'] = (
            climate_df.groupby(['직팜산지코드', '등급코드'])[col].shift(lag)
        )

climate_df = climate_df.drop(columns=climate_cols)

df['yearweek'] = df['year'] * 100 + df['week']
climate_df['yearweek'] = climate_df['yearweek'].astype(int)

df_final = pd.merge(
    df,
    climate_df,
    how='left',
    on=['직팜산지코드', '등급코드', 'yearweek']
)

df_final.drop(columns='yearweek', inplace=True)

df_final.rename(columns={
    'year_x': 'year',
    'week_x': 'week'
}, inplace=True)

df_final.drop(columns=['year_y', 'week_y'], errors='ignore', inplace=True)

print("전체 행:", len(df_final))
print("지연기후 샘플:\n", df_final[[col for col in df_final.columns if '_t-' in col]].head())

✅ 등급 포함 지연기후 병합 완료
전체 행: 132287
지연기후 샘플:
    일평균기온_t-1  최고기온_t-1  최저기온_t-1  평균상대습도_t-1  강수량(mm)_t-1  1시간최고강수량(mm)_t-1  \
0        NaN       NaN       NaN         NaN          NaN               NaN   
1        NaN       NaN       NaN         NaN          NaN               NaN   
2        NaN       NaN       NaN         NaN          NaN               NaN   
3        NaN       NaN       NaN         NaN          NaN               NaN   
4        NaN       NaN       NaN         NaN          NaN               NaN   

   일평균기온_t-2  최고기온_t-2  최저기온_t-2  평균상대습도_t-2  ...  최저기온_t-3  평균상대습도_t-3  \
0        NaN       NaN       NaN         NaN  ...       NaN         NaN   
1        NaN       NaN       NaN         NaN  ...       NaN         NaN   
2        NaN       NaN       NaN         NaN  ...       NaN         NaN   
3        NaN       NaN       NaN         NaN  ...       NaN         NaN   
4        NaN       NaN       NaN         NaN  ...       NaN         NaN   

   강수량(mm)_t-3  1시간최고강수량(mm)_t-3

In [232]:
df_final.rename(columns={
    'year_x': 'year',
    'week_x': 'week'
}, inplace=True)

df_final[['year', 'week', '직팜산지코드'] + [col for col in df_final.columns if '_t-' in col]].head()

,year,week,직팜산지코드,일평균기온_t-1,최고기온_t-1,최저기온_t-1,평균상대습도_t-1,강수량(mm)_t-1,1시간최고강수량(mm)_t-1,일평균기온_t-2,...,최저기온_t-3,평균상대습도_t-3,강수량(mm)_t-3,1시간최고강수량(mm)_t-3,일평균기온_t-4,최고기온_t-4,최저기온_t-4,평균상대습도_t-4,강수량(mm)_t-4,1시간최고강수량(mm)_t-4
0,2018,1,1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,1,1079,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,1,1148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,1,1019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,1,1022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [233]:
df_final.dropna(inplace=True)

df_final = df_final[df_final['총거래량(kg)'] > 0]

print("남은 행 수:", len(df_final))

✅ 결측치 및 거래량 0 제거 완료
남은 행 수: 129501


In [234]:
df_final.to_csv(f"data/{crop_name}_lag처리.csv", index=False, encoding='cp949')

✅ 저장 완료


# 정확도 검증